In [ ]:
import os
import sys
sys.path.append(os.path.join(os.path.abspath(''), "..",".."))

from src.utils import setup_device, setup_seed

local_path = os.path.join(os.path.abspath(''), "..","..", "data", "raw", "Neu", "anomaly")
remote_path = os.path.join(os.path.abspath(''), "..","..", "..", "anomaly")

device, data_path = setup_device(local_path, remote_path)
_init_fn = None #setup_seed()

In [ ]:
import torch
import numpy as np

In [ ]:
model_name = "ProposalRL"
dataset_names = [
    "cassandra",
    "sprout",
    "astaire",
    "backend",
    "homer",
    "homestead",
    "load-balancer",
    "chronos",
    "bono"
]

In [ ]:
from src.preparation.utils import load_datasets
from src.utils import store_results

exclude_anomalies = None

datasets = load_datasets(dataset_names, data_path, exclude_anomalies=exclude_anomalies)

In [ ]:
from src.preparation.transforms import *
import torch_geometric.transforms as T

data_transform_steps = {
    "transforms": [
        MinMaxTransform()
    ],
    "clear_after_use": True
}

In [ ]:
training_config = {
    "extraction_target": "window",
    "epochs": 100,
    "device": device,
    "early_stopping": 30,
    "checkpoint_saving": True,
    "graph_dataset_config": {
        "window_width": 20,
        "sliding_window": 1,
        "flatten_slice": False,        
        "shuffle_settings": {
            "train": True,
            "valid": False,
            "test": False
        },    
        "complete_batches": True,  
        "use_custom_sampler": None,    
        "batch_size": 128,
        "init_fn": _init_fn    
    },
    "augmentate": {
        "proba": 0.0,
        "mu": [0,0],
        "sigma": [0.05, 0.1],
        "offline": True,
        "cat": True
    },
    "optimizer_config": {
        "lr": 1e-3,
        "weight_decay": 1e-5,
        "betas":(0.9, 0.999)
    }
}

In [ ]:
from src.modeling.models_tf import ProposalRL

In [ ]:
from src.modeling.regression_pipeline import RegressionPipeline
from src.modeling.utils import CombinedLoss

In [ ]:
model_settings = {
    "optimizer_class": torch.optim.Adam,
    "loss_class": CombinedLoss,
    "model_class": ProposalRL,
    "save_path": os.path.join(os.path.abspath(''), "..","..", "..", "models", "temp-models")
}

In [ ]:
model_config = {
    "num_node_features": 20,
    "num_hidden": 64,
    "num_flex": 512,
    "dropout": 0.5,
    "nb_polynomial_order": 3,
    "graclus_pooling": True
}

In [ ]:
def score_function(engine):
    val_loss = engine.state.metrics['mse']
    return -val_loss

In [ ]:
pipeline = RegressionPipeline(datasets, 
                        data_transform_steps=data_transform_steps,
                        model_settings=model_settings,
                        model_config=model_config,
                        training_config=training_config,
                        score_function=score_function
                       )

In [ ]:
from src.preparation.splitting import TransferLearningWrapper

# evaluate model & hyperparameters
test_acc, test_loss = pipeline.run(dataset_names, model_name, folds=1, test=False, wrapper=TransferLearningWrapper)

In [ ]:
# store results
store_results(test_acc, 
              test_loss, 
              dataset_names, 
              model_name, [
                  training_config,
                  model_config,
                  model_settings
              ])